In [ ]:
!pip install flask_ngrok
!pip install pyngrok

     |████████████████████████████████| 745 kB 4.3 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=cab218349427df211cbea51fd295ad50e6e1d0f2903b83c6fe579f44de9fe8bd
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
from keras.preprocessing.image import load_img, img_to_array
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
from keras.preprocessing import image
from keras.models import load_model 
from pathlib import Path
import pandas as pd
import tensorflow as tf
import flask
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/CV_1
!ls

Mounted at /content/drive
/content/drive/MyDrive/CV_1
CV_1_research.docx  Flask_Api.ipynb   static
CV_1-st_task.ipynb  fruit_vegetables  templates
example-tSNE.png    ResNet50_FV       Training.docx


In [ ]:
# put your token
!ngrok authtoken 26LPZYNETb2G2mIzWhe9ddHXkYv_6uvhfDYAfqsztiSy9JkZQ

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# Create dataframe for image  generator
def df_img(filepaths):
    
    #Create labels list
    labels = [str(filepaths[i]).split('/')[-2] for i in range(len(filepaths))]
    
    filepath = pd.Series(filepaths, name = 'Filepath').astype(str)
    labels = pd.Series(labels, name = 'Label')
    
    #Create dataframe with filepaths and their labels
    df = pd.concat([filepath, labels], axis =1)
    
    #Shufle dataframe with index reseting
    df = df.sample(frac = 1).reset_index(drop = True)
    
    return df

#Set path for test_dir if used custom model
test_dir = Path('fruit_vegetables/test')
test_filepaths = list(test_dir.glob(r'*/*.*'))
test_df = df_img(test_filepaths)

In [ ]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator( 
    preprocessing_function = tf.keras.applications.resnet50.preprocess_input ) 

test_images = image_generator.flow_from_dataframe(
    dataframe = test_df,
    x_col = 'Filepath',
    y_col = 'Label',
    target_size = (224, 224),
    color_mode = 'rgb',
    class_mode = 'categorical',
    shuffle = False )

Found 359 validated image filenames belonging to 36 classes.


In [ ]:
model = load_model('ResNet50_FV')

In [ ]:
def predict_image(image, custom = False):
  '''
  image: path to your image

  function returns name of predicted class
  '''
  test_image = load_img(image, target_size = (224, 224))
  test_image = img_to_array(test_image)

  test_image = test_image.reshape((1, test_image.shape[0], 
                                   test_image.shape[1], test_image.shape[2]))
  test_image = tf.keras.applications.resnet50.preprocess_input(test_image)

  #model: trained model for classifiactaion
  result = model.predict(test_image)[0].argmax()

  if custom:
    labels = test_images.class_indices
  else:
    labels = {'apple': 0,
 'banana': 1,
 'beetroot': 2,
 'bell pepper': 3,
 'cabbage': 4,
 'capsicum': 5,
 'carrot': 6,
 'cauliflower': 7,
 'chilli pepper': 8,
 'corn': 9,
 'cucumber': 10,
 'eggplant': 11,
 'garlic': 12,
 'ginger': 13,
 'grapes': 14,
 'jalepeno': 15,
 'kiwi': 16,
 'lemon': 17,
 'lettuce': 18,
 'mango': 19,
 'onion': 20,
 'orange': 21,
 'paprika': 22,
 'pear': 23,
 'peas': 24,
 'pineapple': 25,
 'pomegranate': 26,
 'potato': 27,
 'raddish': 28,
 'soy beans': 29,
 'spinach': 30,
 'sweetcorn': 31,
 'sweetpotato': 32,
 'tomato': 33,
 'turnip': 34,
 'watermelon': 35}

  labels = list(labels.items())

  for label, i in labels:
      if i == result:
          return label

In [ ]:
image_folder = os.path.join('static','images/')
app = Flask(__name__)
run_with_ngrok(app) 
app.config["UPLOAD_FOLDER"] = image_folder

In [ ]:
@app.route('/', methods=['GET'])
def home():
  return render_template('index.html')

@app.route('/', methods=['POST'])
def predict():

  imagefile = request.files['imagefile']
  image_path = image_folder + imagefile.filename

  # image will be saved in images folder
  imagefile.save(image_path)

  pic = os.path.join(app.config['UPLOAD_FOLDER'], imagefile.filename)
  #set custom = True if used custom model
  prediction = predict_image(pic, custom = True)

  return render_template('index.html', user_image=pic, prediction_text='The uploaded image is {}'.format(prediction))

In [ ]:
if __name__=='__main__':
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://16d8-35-236-175-102.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [05/May/2022 22:33:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2022 22:33:44] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [05/May/2022 22:34:00] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2022 22:34:00] "GET /static/images/Image_3.jpg HTTP/1.1" 200 -
